# Merge and Reduce Data

The below code performs reduction on the raw QMC data that can be downloaded via the [download_QMC_data.ipynb](./download_QMC_data.ipynb) script. Running this is *not* necessary to reproduce the final plots which appear in the paper.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re,glob,os
import dgutils
from pimcscripts import pimchelp
from collections import defaultdict
import linecache

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

colors = dgutils.colors.get_linear_colors('Spectral',10)
import importlib

In [ ]:
import hg_utils
from hg_utils import raw_data_dir, data_dir, lab, base_dir, gnu_parallel, pimc_bin_path

In [ ]:
ΔR = [0.0,2.0,3.0,4.0]
dirNames = ' '.join([lab(cΔR) for cΔR in ΔR])
T = 2.0

# user should specify which of these they want to do! 
run = {'delete_merge':True, 'merge':True, 'reduce':True, 'copy':True}

# define some hard-coded PIMCIDs for merged data since we will add some raw data into our github repo
pimcid = {}
pimcid[lab(0.0)] = 'a48c7b1e-9699-4db5-b21f-f7eaecb6ffec'
pimcid[lab(2.0)] = 'c98674cb-9818-4827-8ba2-1a996bca1df0'
pimcid[lab(3.0)] = 'b2dcef60-8679-4057-b19a-17bc08d1d397'
pimcid[lab(4.0)] = '1e8b4baa-a2a6-46f4-b0b8-955cec5b317f'

pimcids = ' '.join([pimcid[lab(cΔR)] for cΔR in ΔR])

## Delete the Merged Directory and Re-Generate

In [ ]:
if run['delete_merge']:
    for cΔR in ΔR:
        dirName = f'{base_dir(cΔR,cylinder=False, raw=True)}'
        print(f'Deleting ΔR = {cΔR}: {dirName}')
        ! rm -rf $dirName

## Merge QMC Results over Seeds

In [ ]:
%%time

if run['merge']:
    merge_command = f"parallel --link {pimc_bin_path}/merge.py -T {T} --seeds -s 0.5 --label={{1}} {raw_data_dir}/{{2}}/OUTPUT/ ::: {pimcids} ::: {dirNames}"

    stream = os.popen(merge_command)
    output = stream.read()
    print(output)

## Reduce QMC Results over Temperature

In [ ]:
%%time

if run['reduce']:
    reduce_command = f"parallel {pimc_bin_path}/reduce-one.py -r T -L 100 -u -7.2 {raw_data_dir}/{{}}/OUTPUT/MERGED/CYLINDER/ ::: {dirNames}"
    stream = os.popen(reduce_command)
    output = stream.read()
    print(output)

## Copy Merged and Reduced Data into Repository

In [ ]:
if run['copy']:
    for cΔR in ΔR:
        src_dir = base_dir(cΔR, raw=True)
        dest_dir = base_dir(cΔR)
        !cp $src_dir/*.dat $dest_dir/ 

## Load and Compress Position Data

In [ ]:
match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *[-+?]\ *[0-9]+)?')

for cΔR in ΔR:
    position_file = glob.glob(f'{base_dir(cΔR,cylinder=False, raw=True)}/gce-position-*.dat')[0]
    est_info = linecache.getline(position_file, 2)

    # get the discretization info
    nums = [x for x in re.findall(match_number, est_info)]
    _δ = np.array([float(num) for num in nums[:3]])
    _num_grid_sep = int(nums[-1])

    # load the density and reshape
    _ρ = np.loadtxt(position_file).reshape([_num_grid_sep]*3)

    # save as a compressed .npz file
    np.savez_compressed(f'{base_dir(cΔR,cylinder=False)}/gce-position-{lab(cΔR)}', δ=_δ, ρ=_ρ, num_grid_sep=_num_grid_sep)